# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 17 2022 9:39AM,246670,20,8537531-BO,"ACI Healthcare USA, Inc.",Released
1,Sep 17 2022 9:39AM,246670,20,8537532-BO,"ACI Healthcare USA, Inc.",Released
2,Sep 17 2022 9:39AM,246670,20,8537533-BO,"ACI Healthcare USA, Inc.",Released
3,Sep 17 2022 9:39AM,246670,20,8537534-BO,"ACI Healthcare USA, Inc.",Released
4,Sep 17 2022 9:39AM,246670,20,8537535-BO,"ACI Healthcare USA, Inc.",Released
5,Sep 17 2022 9:39AM,246670,20,8537536-BO,"ACI Healthcare USA, Inc.",Released
6,Sep 17 2022 9:39AM,246670,20,8537537-BO,"ACI Healthcare USA, Inc.",Released
7,Sep 17 2022 9:39AM,246670,20,8537538-BO,"ACI Healthcare USA, Inc.",Released
8,Sep 17 2022 9:39AM,246670,20,8537539-BO,"ACI Healthcare USA, Inc.",Released
9,Sep 17 2022 9:39AM,246670,20,8537540-BO,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,246659,Released,1
14,246660,Released,1
15,246663,Released,1
16,246669,Released,4
17,246670,Released,19


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246659,NaN,NaN,1.0
246660,NaN,NaN,1.0
246663,NaN,NaN,1.0
246669,NaN,NaN,4.0
246670,NaN,NaN,19.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246630,0.0,2.0,1.0
246631,0.0,1.0,1.0
246632,0.0,0.0,2.0
246638,16.0,0.0,1.0
246643,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246630,0,2,1
246631,0,1,1
246632,0,0,2
246638,16,0,1
246643,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246630,0,2,1
1,246631,0,1,1
2,246632,0,0,2
3,246638,16,0,1
4,246643,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246630,,2,1
1,246631,,1,1
2,246632,,,2
3,246638,16,,1
4,246643,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 17 2022 9:39AM,246670,20,"ACI Healthcare USA, Inc."
19,Sep 17 2022 8:50AM,246669,20,"ACI Healthcare USA, Inc."
23,Sep 16 2022 4:35PM,246647,21,"NBTY Global, Inc."
24,Sep 16 2022 4:35PM,246648,21,"NBTY Global, Inc."
25,Sep 16 2022 4:35PM,246659,21,"NBTY Global, Inc."
26,Sep 16 2022 4:35PM,246660,21,"NBTY Global, Inc."
27,Sep 16 2022 4:34PM,246663,21,"NBTY Global, Inc."
28,Sep 16 2022 3:48PM,246657,20,Alumier Labs Inc.
32,Sep 16 2022 3:17PM,246650,15,"Person & Covey, Inc."
47,Sep 16 2022 2:12PM,246643,16,Sartorius Stedim Filters Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 17 2022 9:39AM,246670,20,"ACI Healthcare USA, Inc.",,,19
1,Sep 17 2022 8:50AM,246669,20,"ACI Healthcare USA, Inc.",,,4
2,Sep 16 2022 4:35PM,246647,21,"NBTY Global, Inc.",,,1
3,Sep 16 2022 4:35PM,246648,21,"NBTY Global, Inc.",,,1
4,Sep 16 2022 4:35PM,246659,21,"NBTY Global, Inc.",,,1
5,Sep 16 2022 4:35PM,246660,21,"NBTY Global, Inc.",,,1
6,Sep 16 2022 4:34PM,246663,21,"NBTY Global, Inc.",,,1
7,Sep 16 2022 3:48PM,246657,20,Alumier Labs Inc.,,,4
8,Sep 16 2022 3:17PM,246650,15,"Person & Covey, Inc.",14,,1
9,Sep 16 2022 2:12PM,246643,16,Sartorius Stedim Filters Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 17 2022 9:39AM,246670,20,"ACI Healthcare USA, Inc.",19,,
1,Sep 17 2022 8:50AM,246669,20,"ACI Healthcare USA, Inc.",4,,
2,Sep 16 2022 4:35PM,246647,21,"NBTY Global, Inc.",1,,
3,Sep 16 2022 4:35PM,246648,21,"NBTY Global, Inc.",1,,
4,Sep 16 2022 4:35PM,246659,21,"NBTY Global, Inc.",1,,
5,Sep 16 2022 4:35PM,246660,21,"NBTY Global, Inc.",1,,
6,Sep 16 2022 4:34PM,246663,21,"NBTY Global, Inc.",1,,
7,Sep 16 2022 3:48PM,246657,20,Alumier Labs Inc.,4,,
8,Sep 16 2022 3:17PM,246650,15,"Person & Covey, Inc.",1,,14
9,Sep 16 2022 2:12PM,246643,16,Sartorius Stedim Filters Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 17 2022 9:39AM,246670,20,"ACI Healthcare USA, Inc.",19,,
1,Sep 17 2022 8:50AM,246669,20,"ACI Healthcare USA, Inc.",4,,
2,Sep 16 2022 4:35PM,246647,21,"NBTY Global, Inc.",1,,
3,Sep 16 2022 4:35PM,246648,21,"NBTY Global, Inc.",1,,
4,Sep 16 2022 4:35PM,246659,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 17 2022 9:39AM,246670,20,"ACI Healthcare USA, Inc.",19,NaN,NaN
1,Sep 17 2022 8:50AM,246669,20,"ACI Healthcare USA, Inc.",4,NaN,NaN
2,Sep 16 2022 4:35PM,246647,21,"NBTY Global, Inc.",1,NaN,NaN
3,Sep 16 2022 4:35PM,246648,21,"NBTY Global, Inc.",1,NaN,NaN
4,Sep 16 2022 4:35PM,246659,21,"NBTY Global, Inc.",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 17 2022 9:39AM,246670,20,"ACI Healthcare USA, Inc.",19,0.0,0.0
1,Sep 17 2022 8:50AM,246669,20,"ACI Healthcare USA, Inc.",4,0.0,0.0
2,Sep 16 2022 4:35PM,246647,21,"NBTY Global, Inc.",1,0.0,0.0
3,Sep 16 2022 4:35PM,246648,21,"NBTY Global, Inc.",1,0.0,0.0
4,Sep 16 2022 4:35PM,246659,21,"NBTY Global, Inc.",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
15,493288,2,0.0,30.0
16,246643,1,0.0,0.0
20,1479889,31,3.0,0.0
21,1233277,5,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,15,493288,2,0.0,30.0
1,16,246643,1,0.0,0.0
2,20,1479889,31,3.0,0.0
3,21,1233277,5,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,15,2,0.0,30.0
1,16,1,0.0,0.0
2,20,31,3.0,0.0
3,21,5,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,15,Released,2.0
1,16,Released,1.0
2,20,Released,31.0
3,21,Released,5.0
4,15,Executing,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,15,16,20,21
Status,,,,
Completed,30.0,0.0,0.0,0.0
Executing,0.0,0.0,3.0,0.0
Released,2.0,1.0,31.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,15,16,20,21
0,Completed,30.0,0.0,0.0,0.0
1,Executing,0.0,0.0,3.0,0.0
2,Released,2.0,1.0,31.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,15,16,20,21
0,Completed,30.0,0.0,0.0,0.0
1,Executing,0.0,0.0,3.0,0.0
2,Released,2.0,1.0,31.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()